In [105]:
import time
from selenium import webdriver as wd
from selenium.webdriver import Chrome
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

In [106]:
def scrape_comments(url):
    data1=[]
    data2=[]
    # Warning: need to change the location of the path where you saved the chrome driver
    driver = Chrome(executable_path=r'C:\Program Files\chromedriver.exe')  
    wait = WebDriverWait(driver,5)
    driver.get(url)
    driver.maximize_window()
    time.sleep(5)
    comment_section = driver.find_element_by_xpath('//*[@id="comments"]')

    # Scroll to view the comment section
    driver.execute_script("arguments[0].scrollIntoView();", comment_section)
    time.sleep(5)

    # Scroll all the way down to the bottom
    last_height = driver.execute_script("return document.documentElement.scrollHeight")

    while True:
        # Scroll down 'til "next load"
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")

        # Wait to load everything thus far
        time.sleep(3)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height

    # One last scroll just in case
    driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")


    # Scrape each author name
    for body in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#author-text"))):
        data1.append(body.text)

    # Scrape each comment
    for body in wait.until(EC.presence_of_all_elements_located((By.CSS_SELECTOR, "#content-text"))):
        data2.append(body.text)

    driver.close()

    # Merge each author name with corresponding comment
    mergedData = ['&&&&&&'.join(x) for x in zip(data1,data2)]
    return mergedData
            
# html key for the Id and the comment:
# main
# author-text
# comment

In [120]:
def transform(scrapedData):
    # Convert a list to a dataframe
    df = pd.DataFrame(scrapedData, columns=['chunk'])

    # Add column names
    df[['User ID','Comment']] = df['chunk'].str.split("&&&&&&",expand=True,)

    # Drop unnecessary column(s)
    df = df.drop(columns=['chunk'])
    
    # Save the data as a csv file
    df.to_csv('ScrapedYoutubeComments.csv', index=False)
    print("Showing the first five comments as an example")
    print("The order of the comments shown here may not be the same as what you see in the Youtube comment section")
    print('Go to where you saved this .ipynb file and you will see a new csv file named as "ScrapedYoutubeComments"')
    return df.head()

### Crawler Example:

In [108]:
# Copy the url of the youtube video and paste it in the quotes
url = "https://www.youtube.com/watch?v=uJ7AfPrfqB0&ab_channel=FreeDocumentary-NatureFreeDocumentary-Nature"

In [109]:
# Run this cell to create the csv file!
scrapedData = scrape_comments(url)
transform(scrapedData)

Showing the first five comments as an example
The order of the comments shown here may not be the same as what you see in the Youtube comment section


,User ID,Comment
0,Max Naio,love this God bless whoever watching this in 2021
1,The Nameless Ones,38:51 i love how the penguin and a seal are ju...
2,Jessica Liu,Teen penguins really look like kiwi fruits...
3,Amazing Wildlife Videos 2.0 with Shaun Etsebeth,"Love this, wildlife is what i do. And this mak..."
4,Ali badra Sowe,It's so heartbreaking to watch though at the s...


### Crawler: 

In [101]:
# Copy the url of the youtube video and paste it in the quotes
url = ""

In [102]:
# Run this cell to create the csv file!
scrapedData = scrape_comments(url)
transform(scrapedData)